### **Project - 2**

####**Facility Location Optimization ici inventory and capacity**

Rose Mary Jose

Prepared for: ExxpertSCM Thoughtware Training Pvt Ltd.

Guided by: Mr. Pattabhi Raman

In [ ]:
## importing necessary libraries
import numpy as np
import pandas as pd
import io
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


Facility location optimization

Facility location is the process of identifying the optimal location(s) for facilities such as warehouses, distribution centres, and manufacturing plants. The goal of facility loaction analysis is find optimal location where the weight and distance of the transporting vehicle(tonmiles) is minimum.

About Dataset

In the dataset we take an optimal facility location and find the dustance between the customer locations and the facility location and then we calculate the tonmiles. after optimization the optimal facility location where the tonmiles is mininum


In [ ]:
## reading the dataset
data=pd.read_excel('Facility_location.xlsx')
data

,customer_location,Latitude,Longitude,Load(Ton)
0,Kochi,9.973543,76.296246,7
1,Chennai,13.083592,80.255685,10
2,Bangalore,12.993423,77.568437,15
3,Panaji,15.491269,73.828136,3
4,Pune,18.525901,73.859873,5
5,Hyderabad,17.402453,78.492485,7
6,Indore,22.727118,75.862472,4
7,Lucknow,26.855947,80.953482,5
8,Patna,25.600270,85.143725,3
9,Mumbai,19.092550,72.901081,15


In [ ]:
print('shape of dataset: ',data.shape)
print(data.info())

shape of dataset:  (24, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_location  24 non-null     object 
 1   Latitude           24 non-null     float64
 2   Longitude          24 non-null     float64
 3   Load(Ton)          24 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 896.0+ bytes
None


In [ ]:
data.describe()

,Latitude,Longitude,Load(Ton)
count,24.000000,24.000000,24.000000
mean,22.063395,78.927036,7.250000
std,5.610915,5.120331,3.767712
min,9.973543,72.593458,3.000000
25%,18.320246,75.546105,4.750000
50%,22.876798,77.390795,7.000000
75%,26.303171,82.563037,8.250000
max,31.639363,91.713853,15.000000


In [ ]:
data.isnull().sum()

customer_location    0
Latitude             0
Longitude            0
Load(Ton)            0
dtype: int64

In [ ]:
data.duplicated().sum()

0

In [ ]:
!pip install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [ ]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
## making the list of all columns of the dataframe
customers = data['customer_location'].tolist()
latitudes = data['Latitude'].tolist()
longitudes = data['Longitude'].tolist()
loads = data['Load(Ton)'].tolist()
num_customers = len(customers)

In [ ]:
# Define the Pyomo model
model = pyo.ConcreteModel()

In [ ]:
# Variables
model.lat_optimal = pyo.Var(domain=pyo.Reals)
model.lon_optimal = pyo.Var(domain=pyo.Reals)
model.y = pyo.Var(customers, domain=pyo.Binary)

In [ ]:
# Objective
model.obj = pyo.Objective(
    expr=sum(loads[i] * 111.12*pyo.sqrt((latitudes[i] - model.lat_optimal)**2 + (longitudes[i] - model.lon_optimal)**2)
    for i in range(num_customers)),sense=pyo.minimize)

In [ ]:
# Solve the model using the Ipopt solver
opt = SolverFactory('ipopt')
results = opt.solve(model)

In [ ]:
# Access the optimal latitude and longitude values
optimal_facility_latitude = model.lat_optimal.value
optimal_facility_longitude = model.lon_optimal.value

In [ ]:
min_load_distance = model.obj()
print("Optimal Facility Latitude:", optimal_facility_latitude)
print("Optimal Facility Longitude:", optimal_facility_longitude)
print("Sum of Load * Distance:", min_load_distance)


Optimal Facility Latitude: 22.060033318577428
Optimal Facility Longitude: 77.8418949340904
Sum of Load * Distance: 142294.84376304114


We optimized and find the optimal loaction for facility using pyomo which used ipopt solver. found out that the optimal loaction for our facility/warehouse is 22.0600,77.8418 (Chhuri, Madhya Pradesh), with a load distance of 142294.84 TonKm

In [ ]:
import folium
# Create a map centered at the optimal location
map_center = [optimal_facility_latitude ,optimal_facility_longitude]
map = folium.Map(location=map_center, zoom_start=10)

# Add markers for the customer locations
for i in range(num_customers):
    folium.Marker([data['Latitude'][i], data['Longitude'][i]],
                  popup=f"Customer {i+1}").add_to(map)

# Add a marker for the optimal location
folium.Marker(map_center, popup="Optimal location",
              icon=folium.Icon(color='red')).add_to(map)

# Display the map
map

In [ ]:
## making a new dataset with the load of mumbai replaced with another value
df=data.copy()
index=9
new_value=100
df.at[index,'Load(Ton)'] = new_value
df

,customer_location,Latitude,Longitude,Load(Ton)
0,Kochi,9.973543,76.296246,7
1,Chennai,13.083592,80.255685,10
2,Bangalore,12.993423,77.568437,15
3,Panaji,15.491269,73.828136,3
4,Pune,18.525901,73.859873,5
5,Hyderabad,17.402453,78.492485,7
6,Indore,22.727118,75.862472,4
7,Lucknow,26.855947,80.953482,5
8,Patna,25.600270,85.143725,3
9,Mumbai,19.092550,72.901081,100


In [ ]:
# Define the sets and parameters
customers = df['customer_location'].tolist()
latitudes = df['Latitude'].tolist()
longitudes = df['Longitude'].tolist()
loads = df['Load(Ton)'].tolist()
num_customers = len(customers)


In [ ]:
# Define the Pyomo model
model = pyo.ConcreteModel()

In [ ]:
# Variables
model.lat_optimal = pyo.Var(domain=pyo.Reals)
model.lon_optimal = pyo.Var(domain=pyo.Reals)
model.y = pyo.Var(customers, domain=pyo.Binary)

In [ ]:
# Objective
model.obj = pyo.Objective(
    expr=sum(loads[i] * 111.12* pyo.sqrt((latitudes[i] - model.lat_optimal)**2 + (longitudes[i] - model.lon_optimal)**2)
     for i in range(num_customers)),sense=pyo.minimize)

In [ ]:
# Solve the model using the Ipopt solver
opt = SolverFactory('ipopt')
results = opt.solve(model)

In [ ]:
# Access the optimal latitude and longitude values
optimal_facility_latitude1 = model.lat_optimal.value
optimal_facility_longitude1 = model.lon_optimal.value
min_load_distance = model.obj()

In [ ]:
print("Optimal Facility Latitude:", optimal_facility_latitude1)
print("Optimal Facility Longitude:", optimal_facility_longitude1)
print("Sum of Load * Distance:", min_load_distance)


Optimal Facility Latitude: 19.17785606246781
Optimal Facility Longitude: 73.10209659048975
Sum of Load * Distance: 169817.85622576645


We replaced the value of load of mumbai to 100 from 15 and tried to find the new facility location suggested by our solver and found out that the optimal loaction for our facility/warehouse is 19.177,73.1020 (Kalyan, Maharastra).

In [ ]:
import folium
# Create a map centered at the optimal location
map_center = [optimal_facility_latitude1 ,optimal_facility_longitude1]
map = folium.Map(location=map_center, zoom_start=10)

# Add markers for the customer locations
for i in range(num_customers):
    folium.Marker([df['Latitude'][i], df['Longitude'][i]],
                  popup=f"Customer {i+1}").add_to(map)

# Add a marker for the optimal location
folium.Marker(map_center, popup="Optimal location",
              icon=folium.Icon(color='red')).add_to(map)

# Display the map
map

In [ ]:
df1=df.copy()
index=2
new_value=200
df1.at[index,'Load(Ton)'] = new_value
df1

,customer_location,Latitude,Longitude,Load(Ton)
0,Kochi,9.973543,76.296246,7
1,Chennai,13.083592,80.255685,10
2,Bangalore,12.993423,77.568437,200
3,Panaji,15.491269,73.828136,3
4,Pune,18.525901,73.859873,5
5,Hyderabad,17.402453,78.492485,7
6,Indore,22.727118,75.862472,4
7,Lucknow,26.855947,80.953482,5
8,Patna,25.600270,85.143725,3
9,Mumbai,19.092550,72.901081,100


In [ ]:
# Define the sets and parameters
customers = df1['customer_location'].tolist()
latitudes = df1['Latitude'].tolist()
longitudes = df1['Longitude'].tolist()
loads = df1['Load(Ton)'].tolist()
num_customers = len(customers)

In [ ]:
# Define the Pyomo model
model = pyo.ConcreteModel()

In [ ]:
# Variables
model.lat_optimal = pyo.Var(domain=pyo.Reals)
model.lon_optimal = pyo.Var(domain=pyo.Reals)
model.y = pyo.Var(customers, domain=pyo.Binary)

In [ ]:
# Objective
model.obj = pyo.Objective(
    expr=sum(loads[i] * 111.12* pyo.sqrt((latitudes[i] - model.lat_optimal)**2 + (longitudes[i] - model.lon_optimal)**2) for i in range(num_customers)),
    sense=pyo.minimize)

In [ ]:
# Solve the model using the Ipopt solver
opt = SolverFactory('ipopt')
results = opt.solve(model)

  - termination condition: maxIterations
  - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations Exceeded.


In [ ]:
# Access the optimal latitude and longitude values
optimal_facility_latitude2 = model.lat_optimal.value
optimal_facility_longitude2 = model.lon_optimal.value

In [ ]:
min_load_distance = model.obj()
print("Optimal Facility Latitude:", optimal_facility_latitude1)
print("Optimal Facility Longitude:", optimal_facility_longitude1)
print("Sum of Load * Distance:", min_load_distance)


Optimal Facility Latitude: 19.17785606246781
Optimal Facility Longitude: 73.10209659048975
Sum of Load * Distance: 270715.39023905725


In [ ]:
import folium
# Create a map centered at the optimal location
map_center = [optimal_facility_latitude2 ,optimal_facility_longitude2]
map = folium.Map(location=map_center, zoom_start=10)

# Add markers for the customer locations
for i in range(num_customers):
    folium.Marker([df1['Latitude'][i], df1['Longitude'][i]],
                  popup=f"Customer {i+1}").add_to(map)

# Add a marker for the optimal location
folium.Marker(map_center, popup="Optimal location",
              icon=folium.Icon(color='red')).add_to(map)

# Display the map
map